# OOPAO: The Python Legacy of OOMAO


The list of Adaptive Optics (AO) simulators in the community has constantly been growing, guided by different needs and purposes (Compass, HCIPY, OOMAO, SOAPY, YAO…). In this paper, we present OOPAO (Object Oriented Python Adaptive Optics), a simulation tool based on the Matlab distribution OOMAO to adapt its philosphy to the Python language. This code was initially intended for internal use but the choice was made to make it public as it can benefit the community since it is fully develloped in Python. The OOPAO repository is available in free access on GitHub ( https://github.com/cheritier/OOPAO) with several tutorials. 
The tool consists of a full end-to-end simulator designed for AO analysis purposes. The principle is that the light from a given light source can be propagated through multiple objects (Atmosphere, Telescope, Deformable Mirror, Wave-Front Sensors...) among which experimental features can be input, in the spirit of OOMAO.
This paper provides an overview of the current capabilities of the code and can be used as a user manuel for interested users.

## Getting started with OOPAO

The source code of OOPAO is available on GitHub https://github.com/cheritier/OOPAO. This page provides all the informations necessary to install the required packages. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import OOPAO

# How To Use OOPAO
This section offers an overview of the main functionalities of the OOPAO code to present the philosophy of the code. 

## The Source Object
### Initialization
The source object carries the wavelength information as well as the flux information
It is initialized specifying the optical band and the magnitude of the star.  

In [ ]:
from OOPAO.Source import Source
# to create a natural guide star in I band of magnitude 5
ngs = Source(magnitude = 5,
             optBand   ='I')

The optical bands are defined within the `Source` object inside the `photometry` method. 
The default photometry have been imported from the OOMAO toolkit. You can edit this function to add your own optical bands. 
```
                   [ Wavelength , Bandwidth, Zero-Point ]
        phot.U      = [ 0.360e-6 , 0.070e-6 , 2.0e12 ]
        phot.V      = [ 0.550e-6 , 0.090e-6 , 3.3e12 ]
        ...
```

### Properties
You can access a few properties of the ```ngs``` object: 

In [ ]:
print('The source Wavelenght is '+str(ngs.wavelength) + ' [m] ')
print('The source Magnitude is '+str(ngs.magnitude))
print('The source Altitude is '+str(ngs.altitude) + ' [m] ')
print('The source Coordinates are '+str(ngs.coordinates) + ' [arcsec,deg] ')
print('The source Flux is '+str(ngs.nPhoton) + ' [phot/m2/s] ')
print('The source Type is '+str(ngs.type))
print('The source Tag is '+str(ngs.tag))

The main properties of an OOPAO class can be displayed simply entering its name in a terminal.

## The Telescope Object
You can import the Telescope class from OOPAO using:

In [ ]:
from OOPAO.Telescope import Telescope

The Telescope is a central object in OOPAO, it is required to initialize many of the OOPAO classes as it carries the pupil definition and pixel size.

A Source object is associated to the Telescope that carries the flux and wavelength information. 

An Atmosphere object can be paired to the Telescope to propagate the light through turbulent phase screens.

To initialize a Telescope object, you need to specify a few parameters, the diameter and the resolution of the telescope. The pixel size associated will define the pixel size of the phase-screens.

The sampling of the atmospheric phase screens is driven by different factors: 
- **Sampling of the Wave-Front Sensor sub-apertures:** the WFS object requires a resolution that is `n` times the number of subaperture where `n` should be an even number $>=$ 4.

- **Sampling of the turbulence:** It is recommanded to use at least 3 pixels per $r_0$, the Fried Parameter expressed at the wavelength of interest. 
- **Sampling of the influence functions of the Deformable Mirror:** Two pixels per influence functions is typically not enough and could lead to numerical errors when trying to apply a shape on the Deformable Mirror. 

In [ ]:
# telescope parameters
sensing_wavelength = ngs.wavelength      # sensing wavelength of the WFS, read from the ngs object
n_subaperture      = 20                  # number of subaperture accross the diameter
diameter           = 8                   # diameter of the support of the phase screens in [m]
resolution         = n_subaperture*8     # resolution of the phase screens in pixels
pixel_size         = diameter/resolution # size of the pixels in [m]
obs_ratio          = 0.1                 # central obstruction in fraction of the telescope diameter
sampling_time      = 1/1000              # sampling time of the AO loop in [s]

# initialize the telescope object
tel = Telescope(diameter          = diameter,
               resolution         = resolution,
               centralObstruction = obs_ratio,
               samplingTime       = sampling_time)



We can notice that, as indicated, so far no light is propagated through the telescope. See the section **Propagating light to the Telescope**


### Properties

A few properties of the `tel` object can be accessed:
* `tel.pupil` The pupil mask of the telescope
* `tel.pixelArea` The number of valid pixels in the pupil
* `tel.pupilReflectivity` User-defined pupil reflectivity (default: uniform)

### Methods
####  Apply spider to the pupils
You can defin a list of angle for each spider of the telescope. A spider is defined from the center to the edge of the pupil support. If necessqry, a shift offset can be applied to the spiders.


In [ ]:
n_spider = 4
spider_angle = np.linspace(0+30,360+30,n_spider,endpoint=False)
tel.apply_spiders(spider_angle,thickness_spider=0.05,offset_X=[0.15,-0.15,-0.15,0.15],offset_Y=[0.15,0.15,-0.15,-0.15])
pupil_spiders =  tel.pupil.copy()
# reset to initial pupil
tel.apply_spiders(angle = [0],thickness_spider=0)
plt.subplot(1,2,1),plt.imshow(pupil_spiders),plt.axis('off'),plt.title('Spiders')
plt.subplot(1,2,2),plt.imshow(tel.pupil),plt.axis('off'),plt.title('No Spiders');

### Propagating light to the Telescope
#### Coupling a Source to the Telescope
To propagate the light of the Source through the Telescope, it is necessary to attach the `ngs` object to the `tel` object using the `*` operator.

In [ ]:
ngs*tel

This time, the bottom part indicated that the NGS has been properly propagated to the telescope.
The optical path can be displayed using the `print_optical_path()` method 

In [ ]:
tel.print_optical_path()

The `tel` object now has an `src` attribute that corresponds to the `ngs` object


#### Flux properties
The flux properties are now adapted to the pupil of the telescope. 
The `tel.src` has a `fluxMap` property that displays a 2D map of the **number of photons per pixel per temporal iteration** 

#### Access the wave-front properties
The Optical Path Difference in [m] of the wave-front can be accessed using the `tel.OPD`

The Phase in [rad] wave-front can be accessed using the `ngs.phase` or the `tel.src.phase`


### Compute the PSF of the system

In [ ]:
# set back the reflectivity 1
tel.pupilReflectivity = tel.pupil.astype(float)
ngs*tel

# compute PSF method
zeroPaddingFactor = 6
tel.computePSF(zeroPaddingFactor=zeroPaddingFactor)

# get current pixel scale
size_pixel_arcsec = 206265*(tel.src.wavelength/tel.D)/zeroPaddingFactor
N                 = 300
normalized_PSF    = tel.PSF[N:-N,N:-N]# crop to zoom on the center
fov               = normalized_PSF.shape[0]*size_pixel_arcsec # get the corresponding fov
plt.imshow(np.log10(normalized_PSF),extent=[-fov/2,fov/2,-fov/2,fov/2])
plt.clim([-6,4]),plt.xlabel('arcsec'),plt.ylabel('arcsec');

## The Atmosphere Object
You can import the Telescope class from OOPAO using:

In [ ]:
from OOPAO.Atmosphere import Atmosphere

An Atmosphere is made of one or several layer of turbulence that follow the Van Karmann statistics. Each layer is considered to be independant to the other ones and has its own properties (direction, speed, etc.)

The Atmosphere object can be defined for a single Source object (default) or multi Source Object (see Asterism class). The Source coordinates allow to span different areas in the field (defined as well by tel.fov).

If the source type is an LGS the cone effect is considered using an interpolation. 

NGS and LGS can be combined together in the Asterism object. 

The convention chosen is that all the wavelength-dependant atmosphere parameters are expressed at 500 nm. 


In [ ]:
# setting a two layers atmosphere
atm = Atmosphere(telescope      = tel,
                 r0             = 0.15,
                 L0             = 25,
                 fractionalR0   = [0.7, 0.3  ],
                 altitude       = [0  , 10000],
                 windDirection  = [0  , 20   ],
                 windSpeed      = [5  , 10   ])

The atmosphere has to be initialized using the `atm.initializeAtmosphere()` command

In [ ]:
atm.initializeAtmosphere(telescope=tel)

The `atm` layers can be displayed using the `atm.display_atm_layers()` command

In [ ]:
atm.display_atm_layers()

The atmosphere `atm` can be combined to the telescope using the `+`. In that case, the OPD of the telescope is copied from the atmosphere OPD:

In [ ]:
tel+atm
OPD_tel_paired = tel.OPD.copy()
tel-atm
OPD_tel_separated = tel.OPD.copy()
plt.subplot(1,2,1),plt.imshow(OPD_tel_paired),plt.axis('off'),plt.title('Telescope OPD (Paired)')
plt.subplot(1,2,2),plt.imshow(OPD_tel_separated),plt.axis('off'),plt.title('Telescope OPD (Separated)');

# Deformable Mirror


The deformable mirror is mainly characterized with its influence functions. They can be user-defined and loaded in the model but the default case is a cartesian DM with gaussian influence functions and normalized to 1 [m]. 
The DM is always defined in the pupil plane and can be conjugated to different altitude. 
### Fried-Geometry
By default, the Deformable Mirror geometry follows the Fried's definition where each actuator is located at the corner of 4 WFS sub-aperture. The number of sub-aperture is then a required parameter of the Deformable Mirror class. 

In [ ]:
from OOPAO.DeformableMirror import DeformableMirror
mechanical_coupling = 0.45

dm_fried = DeformableMirror(  telescope    = tel,
                        nSubap       = n_subaperture, # by default n_subaperture+1 actuators are considered (Fried Geometry)
                        mechCoupling = mechanical_coupling)

The DM commands can be applied as a linear combination of the influence functions `dm.modes` setting the `dm.coefs` property:

In [ ]:
dm_fried.coefs = np.random.rand(dm_fried.nValidAct)

The light can be propagated through the telescope, passing by the deformable mirror using the `*` operator:


In [ ]:
ngs*tel*dm_fried
plt.subplot(1,2,1),plt.imshow(tel.OPD),plt.axis('off'),plt.title('Telescope OPD')
plt.subplot(1,2,2),plt.imshow(dm_fried.OPD),plt.axis('off'),plt.title('Deformable Mirror OPD');

### Mis-Registration
To apply a mis-registration

By default, the Deformable Mirror geometry follows the Fried's definition where each actuator is located at the corner of 4 WFS sub-aperture. The number of sub-aperture is then a required parameter of the Deformable Mirror class.


In [ ]:
from OOPAO.MisRegistration import MisRegistration

m = MisRegistration()
m.shiftX = 0.05*tel.D/n_subaperture
m.shiftY = 0.1*tel.D/n_subaperture
m.rotationAngle = 15
m.radialScaling = 0.1
m.tangentialScaling = 0
m.anamorphosisAngle = 45

dm_mis_registered = DeformableMirror(  telescope    = tel,
                        nSubap       = n_subaperture, # by default n_subaperture+1 actuators are considered (Fried Geometry)
                        mechCoupling = mechanical_coupling,
                        misReg       = m)

### User-defined geometry


In [ ]:
# create a circular DM
n_ring = 11; n_start = 5; cx = []; cy = []
for i in range(1,n_ring+1):
    r = i*(tel.D/2)/(n_ring)
    theta = np.linspace(0,2*np.pi,n_start,endpoint=False)
    cx = np.hstack((cx,(r*np.cos(theta))))
    cy = np.hstack((cy,(r*np.sin(theta))))
    n_start += 6
# input coordinates must be provided in a 2D array [nAct,2]
circular_coordinates = np.vstack((cx,cy)).T
dm_circular = DeformableMirror(  telescope    = tel,
                        nSubap       = n_subaperture, # by default n_subaperture+1 actuators are considered (Fried Geometry)
                        mechCoupling = mechanical_coupling,coordinates=circular_coordinates)

### User-Defined influence functions


It is possible to input the influence functions of the DM directly. In this example, we consider a modal DM that corresponds to zernike polynomials. 


In [ ]:
from OOPAO.Zernike import Zernike
Z = Zernike(tel,300) # create a Zernike object considering 300 polynomials
Z.computeZernike(tel) # compute the Zernike
Z_2D = Z.modesFullRes.reshape((tel.resolution**2,Z.nModes))# reshape the 3D cube in 2D for the dm class
dm_Zernike = DeformableMirror( telescope = tel,
                               nSubap    = n_subaperture, # keep it to n_subaperture ( required to apply mis-registrations)
                               modes     = Z_2D, print_dm_properties=False)
plt.subplot(1,2,1),plt.imshow(dm_fried.modes[:,40].reshape(tel.pupil.shape)),plt.axis('off'),plt.title('Zonal DM influence function')
plt.subplot(1,2,2),plt.imshow(dm_Zernike.modes[:,40].reshape(tel.pupil.shape)),plt.axis('off'),plt.title('Modal DM influence function');

We can display the cube of the influence functions to display the position of the actuators with respect to the pupil. 

In [ ]:
cube_IF_fried    = np.reshape(np.sum(dm_fried.modes**6,axis =1),[tel.resolution,tel.resolution])
cube_IF_mis_reg  = np.reshape(np.sum(dm_mis_registered.modes**6,axis =1),[tel.resolution,tel.resolution])
cube_IF_circular = np.reshape(np.sum(dm_circular.modes**6,axis =1),[tel.resolution,tel.resolution])

plt.subplot(1,3,1),plt.imshow(cube_IF_fried/cube_IF_fried.max() +0.5*tel.pupil),plt.axis('off'),plt.title('Fried Geometry')
plt.subplot(1,3,2),plt.imshow(cube_IF_mis_reg/cube_IF_mis_reg.max() +0.5*tel.pupil),plt.axis('off'),plt.title('Misregistration');
plt.subplot(1,3,3),plt.imshow(cube_IF_circular/cube_IF_circular.max() +0.5*tel.pupil),plt.axis('off'),plt.title('User-Defined Geometry');

# Wave-Front Sensors

## Pyramid WFS
The pyramid object consists mainly in defining the PWFS phase mask to apply the filtering of the electro-magnetic field. 
Many parameters can allow to tune the pyramid model:
* Number of pixels `nSubap` along the telescope pupil support. `tel.resolution` should be a multiple of `nSubap` and >=4 
* Cicular modulation radius in λ/D `modulation`. By default the number of modulation points ensures to have one point every λ/D on the circular trajectory but this sampling can be modified by the user. The number of modulation points is a multiple of 4 to ensure that each quadrant has the same number of modulation points. A user-defined modulation path can be input.
* The type of post-processing of the PWFS signals (slopes-maps, full-frame,etc). To be independent from this choice, the pyramid signals are named “wfs.signal_2D” for either the Slopes-Maps or the camera frame and `signal` for the signal reduced to the valid pixels considered.
* The intensity threshold to select the valid pixel 
* Centering of the mask and of the FFT on 1 or 4 pixels `psfCentering`
* Number of pixels separating the PWFS pupils `n_pix_separation`
* Number of pixels on the edge of the Pyramid pupils`n_pix_edge`
* The modulation value for the calibration and selection of the valid pixels `calibModulation` 

In addition, the Pyramid object has a Detector object as a child-class that provides the pyramid signals. It can be access through `pwfs.cam`

In [ ]:
from OOPAO.Pyramid import Pyramid

pwfs = Pyramid(telescope        = tel,
               nSubap           = n_subaperture,
               modulation       = 3,
               lightRatio       = 0.1,
               postProcessing   = 'slopesMaps',
               n_pix_separation = 10,
               n_pix_edge       = 5)
plt.figure(), 
plt.subplot(1,2,1),plt.imshow(pwfs.m),plt.title('PWFS mask') # 2D phase mask
plt.subplot(1,2,2),plt.imshow(pwfs.cam.frame),plt.title('PWFS Camera Frame'); # camera detector frame

It is possible to shift the Pyramid pupils using the `apply_shift_wfs` method by applying an additional Tip/Tilt on each quadrant of the phase mask

In [ ]:
pwfs.apply_shift_wfs(sx=[5,0,2,0],sy=[0,0,4,0])
plt.figure(), 
plt.subplot(1,3,1),plt.imshow(pwfs.m),plt.axis('off'), plt.title('New PWFS mask') # 2D phase mask
plt.subplot(1,3,2),plt.imshow(pwfs.m-pwfs.initial_m),plt.axis('off'), plt.title('Differential mask') # 2D phase mask
plt.subplot(1,3,3),plt.imshow(pwfs.cam.frame),plt.axis('off'), plt.title('Camera Frame'); # camera detector frame
# set it back to default
pwfs.apply_shift_wfs(sx=0,sy=0)


The focal plane electric field is accessible through the `modulation_camera_em` property. 
This is used by the `get_modulation_frame` method to compute the intensity in at the focal plane, afte the Tip/Tilt modulation mirror. The `radius` allows to select the field of view.

In [ ]:
# acquire a frame with the focal plane camera
pwfs*pwfs.focal_plane_camera
plt.figure(), 
plt.subplot(1,2,1),plt.imshow(pwfs.focal_plane_camera.frame)
plt.axis('off'), plt.title('PWFS Focal Plane'); # radius in lambda/D


## Shack Hartmann WFS

A Shack Hartmann object consists in defining a 2D grid of lenslet arrays located in the pupil plane of the telescope to estimate the local tip/tilt seen by each lenslet. 
By default the Shack Hartmann detector is considered to be noise-free (for calibration purposes). These properties can be switched on and off on the fly (see properties)
        It requires the following parameters: 

* The number of lenslets `nSubap` accross the support of the telescope pupil 
* The intensity threshold `lightRatio` to select the valid lenslets
* The flag `is_geometric` to switch from diffractive to geometric Shack Hartmann
* `shannon_sampling` to sample the diffractive spots with 2 pixels per lambd/D (`True`) or 1 pixel per lambda/D (`False`)
* theshold to compute the center of gravity `threshold_cog` with respect to the maximum value of the image



In [ ]:
from OOPAO.ShackHartmann import ShackHartmann
shwfs = ShackHartmann(telescope    = tel,
              nSubap             = n_subaperture,
              lightRatio         = 0.5,
              is_geometric       = False, 
              shannon_sampling   = True,
              threshold_cog      = 0.1)
plt.figure(), 
plt.subplot(1,2,1),plt.imshow(shwfs.valid_subapertures),plt.title('SH-WFS Valid lenslets') # 2D phase mask
plt.subplot(1,2,2),plt.imshow(shwfs.cam.frame),plt.title('SH-WFS Camera Frame'); # camera detector frame

it is possible to switch from geometric to diffractive shack hartmann simply by setting `wfs.is_geometric` to `True` or `False`

In [ ]:
# make sure that the telescope OPD is initiliazed
tel.resetOPD()
ngs*tel*shwfs
shwfs.is_geometric = True

# Modal Basis
A few modal basis or available in OOPAO: Zernike and KL Modes.
## Zernike Polynomials

In [ ]:
from OOPAO.Zernike import Zernike
Z = Zernike(tel,9)      # first initialize the Zernike object
Z.computeZernike(tel)   # compute the Zernike for the desired telescope
plt.figure()
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(Z.modesFullRes[:,:,i]),plt.axis('off'), plt.title(Z.modeName(i));

The Zernike can be projected on the `dm`using the dm influence functions `modes` from the `dm` object

In [ ]:
M2C_zernike = np.linalg.pinv(np.squeeze(dm_fried.modes[tel.pupilLogical,:]))@Z.modes

It is possible to display the modal basis using the `displayMap` function

In [ ]:
from OOPAO.tools.displayTools import displayMap
dm_fried.coefs = M2C_zernike[:,:9]
ngs*tel*dm_fried
displayMap(tel.OPD)

## KL Modal Basis
A simplified function is available to provide KL modes orthogonal in the DM space, forcing the Tip/Tilt to be included in the Modal basis. 

A more detailed fonctions is available in the `compute_KL_modal_basis` if required.

In [ ]:
from OOPAO.calibration.compute_KL_modal_basis import compute_KL_basis
M2C_KL = compute_KL_basis(tel=tel,atm =atm,dm=dm_fried)

In [ ]:
dm_fried.coefs = M2C_KL[:,:9]
ngs*tel*dm_fried
displayMap(tel.OPD)

# Calibration
The measurement of the interaction matrix is done using the `InteractionMatrix` class

In [ ]:
tel.display_optical_path = False
from OOPAO.calibration.InteractionMatrix import InteractionMatrix

# modal interaction matrix
calib_modal = InteractionMatrix( ngs      = ngs,                    
                                 atm            = atm,                   
                                 tel            = tel,              
                                 dm             = dm_fried,         
                                 wfs            = shwfs,            
                                 M2C            = M2C_KL[:,:300],   # M2C matrix to use for the calibratio
                                 stroke         = 1e-9,             # stroke for the calibration in [nm]
                                 nMeasurements  = 1,               # number of measurements in parallel
                                 noise          = 'off')            # enable noise during the calibration

The details of the pseudo inverse of the interaction matrix are accessible within the properties of `calib_modal`. 

The input matrix is denoted  `calib_modal.D` and its pseudo inverse  `calib_modal.M`. 

By default, no singular values are truncated in the pseudo inverse: 
$M = (D^T.D)^{-1}D^T$

It is possible to recompute `calib_modal.M` truncating singular values using the `calib_modal.nTrunc` property:

In [ ]:
# truncate the 10 last singular values 
calib_modal.nTrunc = 10

## Reconstructing  a phase from WFS measurements
It is now possible to reconstruct a given phase using wfs measurements using the `calib_modal.M` matrix:



In [ ]:
input_modes = np.random.randn(300)*1e-9
dm_fried.coefs = M2C_KL[:,:300]@ input_modes
ngs*tel*dm_fried*shwfs

plt.plot(input_modes,label='Input')
plt.plot(calib_modal.M@shwfs.signal,'o',label='Reconstructed')
plt.xlabel(''),plt.legend();

# Closed Loop Tutorial
This is a simple closed-loop tutorial. To get the visual simulation with a GUI, make use of the tutorials codes in a simulation environment that allows interactive display.

In [ ]:
# Define a scientific source: 
src = Source('K',10)
tel.resetOPD()  # initialize Telescope
dm = dm_fried
dm_fried.coefs=0      # initialize DM commands

tel+atm # combine tel and atm to enable turbulence

tel.computePSF(4)
   
# These are the calibration data used to close the loop
calib_CL    = calib_modal
M2C_CL      = M2C_KL[:,:300]

# initialize DM commands
ngs*tel*dm*shwfs

nLoop = 200

# allocate memory to save data
SR                      = np.zeros(nLoop)
total                   = np.zeros(nLoop)
residual                = np.zeros(nLoop)
wfsSignal               = np.arange(0,shwfs.nSignal)*0

# loop parameters
gainCL                  = 0.4
display                 = True

reconstructor = M2C_CL@calib_CL.M

for i in range(nLoop):
    # update phase screens => overwrite tel.OPD and consequently tel.src.phase
    atm.update()
    # save phase variance
    total[i]=np.std(tel.OPD[np.where(tel.pupil>0)])*1e9
    # save turbulent phase
    turbPhase = tel.src.phase
    # propagate to the WFS with the CL commands applied
    ngs*tel*dm*shwfs
    # propagate to the source with the CL commands applied
    src*tel
    dm.coefs=dm.coefs-gainCL*np.matmul(reconstructor,wfsSignal)
    # store the slopes after computing the commands => 2 frames delay
    wfsSignal=shwfs.signal
    # store data   
    SR[i]=np.exp(-np.var(tel.src.phase[np.where(tel.pupil==1)]))
    residual[i]=np.std(tel.OPD[np.where(tel.pupil>0)])*1e9
    print('Loop'+str(i)+'/'+str(nLoop)+' AO residual: '+str(residual[i])+' nm')

# plot when done
plt.figure()
plt.plot(total)
plt.plot(residual)